In [17]:
import pandas as pd
import numpy as np
import sklearn

# Load data

In [18]:
all_df = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
item_info_df = pd.read_csv('ml-100k/u.item', sep='|', names=['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'], encoding='latin-1')
users = all_df.user_id.unique()
items = all_df.item_id.unique()
n_users = users.shape[0]  
n_items = items.shape[0]


In [19]:
all_df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [20]:
item_info_df

,item_id,title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,childrens,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# split the data into training and test sets
train_df = all_df.sample(frac=0.75, random_state=42)
test_valid_df = all_df.drop(train_df.index)
valid_df = test_valid_df.sample(frac=0.4, random_state=42)
test_df = test_valid_df.drop(valid_df.index)

# Features

In [22]:
from itertools import combinations

# item similarity matrix

In [23]:
from sklearn.metrics.pairwise import pairwise_distances

In [24]:
train_ui_matrix = np.zeros((n_users, n_items))
for row in train_df.itertuples(index=False):
    train_ui_matrix[row[0]-1, row[1]-1] = row[2]

valid_ui_matrix = np.zeros((n_users, n_items))
for row in valid_df.itertuples(index=False):
    valid_ui_matrix[row[0]-1, row[1]-1] = row[2]
    
test_ui_matrix = np.zeros((n_users, n_items))
for row in test_df.itertuples(index=False):
    test_ui_matrix[row[0]-1, row[1]-1] = row[2]

In [25]:
item_similarity = np.ones((n_items, n_items)) - pairwise_distances(train_ui_matrix.T, metric='cosine')

# genre similarity matrix

In [26]:
genre = item_info_df.drop(['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url'], axis=1).to_numpy()

In [27]:
genre_similarity = np.ones((n_items, n_items)) - pairwise_distances(genre, metric='cosine')

In [28]:
pairwise_distances(genre, metric='cosine')

array([[0.        , 1.        , 1.        , ..., 1.        , 0.42264973,
        1.        ],
       [1.        , 0.        , 0.42264973, ..., 1.        , 1.        ,
        1.        ],
       [1.        , 0.42264973, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        0.29289322],
       [0.42264973, 1.        , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.29289322, 1.        ,
        0.        ]])

# Bipartite Graph

In [29]:
import networkx as nx
from networkx.algorithms import bipartite

In [30]:
beta = 0.8

In [31]:
UIG = nx.Graph()
UIG.add_nodes_from(users, bipartite=0)
UIG.add_nodes_from(items, bipartite=1)
edges = all_df[['user_id', 'item_id']].values
UIG.add_edges_from(edges)

In [32]:
IG = bipartite.overlap_weighted_projected_graph(UIG, items)
IG_edges = list(IG.edges(data=True))

In [33]:
IG_matrix = np.zeros((n_items, n_items))
for item_id_0, item_id_1, weight in IG_edges:
    IG_matrix[item_id_0 - 1, item_id_1 - 1] = weight['weight']
    IG_matrix[item_id_1 - 1, item_id_0 - 1] = weight['weight']

# Transition Matrix

In [34]:
transition_matrix_item = np.zeros((n_items, n_items))
transition_matrix_BG = np.zeros((n_items, n_items))
transition_matrix_genre = np.zeros((n_items, n_items))
for i in range(n_items):
    sim_sum = item_similarity[i].sum()
    genre_sum = genre_similarity[i].sum()
    bg_sum = IG_matrix[i].sum()
    for j in range(n_items):
        transition_matrix_item[i,j] = item_similarity[i][j]/sim_sum
        transition_matrix_BG[i,j] = IG_matrix[i][j]/bg_sum
        transition_matrix_genre[i,j] = genre_similarity[i][j]/genre_sum

# Random walk frame work

In [35]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm import tqdm

In [56]:
def random_walk_predict(ratings, transition_matrix, alpha):
    p_tilde = np.linalg.pinv(np.eye(n_items)-alpha*transition_matrix)
    final_rating = alpha*np.dot(ratings, np.dot(transition_matrix, p_tilde))
    return final_rating

In [48]:
def rmse(prediction, test_data_matrix):
    mask = test_data_matrix.nonzero()
    prediction = prediction[mask].flatten()
    truth = test_ui_matrix[mask].flatten()
    return sqrt(mean_squared_error(prediction, truth))

## parameter selection

In [44]:
alphas = [0.7, 0.8, 0.9, 1.0]
betas = [0.6, 0.7, 0.8, 0.9, 1.0]
w1s = w2s = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
weights = [(w1, w2) for w1 in w1s for w2 in w2s if w1 + w2 <= 1 ]
parameters = [(alpha, beta, w1, w2) for alpha in alphas for beta in betas for w1, w2 in weights]

In [45]:
results = {}
for parameter in tqdm(parameters):
    alpha, beta, w1, w2 = parameter
    transition_matrix = beta*(w1*transition_matrix_item + w2*transition_matrix_BG + (1-w1-w2)*transition_matrix_genre) + (1-beta)/n_items
    prediction = random_walk_predict(train_ui_matrix, transition_matrix, alpha)
    results[parameter] = rmse(prediction, valid_ui_matrix)
    

  6%|▌         | 74/1320 [05:43<1:36:27,  4.65s/it]


KeyboardInterrupt: 

In [57]:
# %%timeit
alpha, beta, w1, w2 = 0.8, 0.8, 0.6, 0.2
prediction = random_walk_predict(train_ui_matrix, transition_matrix, alpha)
result = rmse(prediction, test_ui_matrix)

4.78 s ± 87.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
print('rmse for random walk: {}'.format(result))

rmse for random walk: 1.2572757224291555
